In [1]:
import os
import datetime
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

log_pr = pd.read_pickle("../data/log_price.df")
volu = pd.read_pickle("../data/volume_usd.df")

## Feature Engineering

In [2]:
features = log_pr.reset_index().melt(id_vars=['timestamp'])
features.columns = ['timestamp', 'stock', 'log_pr']
features = features.set_index(['timestamp', 'stock']).sort_index()

### backward returns

In [3]:
for i in [3, 5, 10, 20]:
    features['log_pr_{}'.format(i)] = -features.groupby(level='stock').log_pr.diff(i)

In [5]:
temp = log_pr.rolling(10).mean()
temp.head(10)

,0,1,2,3,4,5,6,7,8,9
timestamp,,,,,,,,,,
2021-07-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-01 00:01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-01 00:02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-01 00:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-01 00:04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-01 00:05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-01 00:06:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-01 00:07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-01 00:08:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### log volume and volume backward diffrence

In [6]:
volu_df = volu.reset_index().melt(id_vars=['timestamp'])
volu_df.columns = ['timestamp', 'stock', 'volu']
volu_df = volu_df.set_index(['timestamp', 'stock']).sort_index()
volu_df.head(10)

volu
timestamp  stock              
2021-07-01 0      1.475069e+05
           1      5.808116e+05
           2      1.860058e+04
           3      1.601353e+05
           4      2.765960e+05
           5      2.916289e+05
           6      8.544255e+05
           7      1.801509e+05
           8      4.151507e+06
           9      7.568141e+06

In [7]:
log_fn = lambda x: np.log(x+1)
features['log_volu'] = volu_df.groupby(level='stock').volu.apply(log_fn)

In [8]:
for i in [1]:
    features['log_volu_{}'.format(i)] = features.groupby(level='stock').log_volu.diff(i)
    # features['volu_{}',format(i)] = volu_df.groupby(level='stock').volu.diff(i)

### Exponential Moving average


In [12]:
for i in [10, 30, 50]:
    ema = lambda x: x.ewm(span=i).mean()
    features['pr_ema_{}'.format(i)] = features.groupby(level='stock').log_pr.apply(ema)

### RSI

In [23]:
def rsi(close_delta, periods=20, ema=True):
    """
    Returns a pd.Series with the relative strength index.
    """
    close_delta = close_delta.diff()

    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    
    if ema == True:
	    # Use exponential moving average
        ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    else:
        # Use simple moving average
        ma_up = up.rolling(window = periods, adjust=False).mean()
        ma_down = down.rolling(window = periods, adjust=False).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    return rsi

In [26]:
features['rsi_20'] = features.groupby(level='stock').log_pr.apply(rsi)
features['rsi_30'] = features.groupby(level='stock').log_pr.apply(rsi, periods=30)
features['rsi_50'] = features.groupby(level='stock').log_pr.apply(rsi, periods=50)

### Moving std. and Range of Oscillation

In [27]:
# std_5 = lambda x: x.rolling(5).std()
std_10 = lambda x: x.rolling(10).std()
features['pr_3_std_10'] = features.groupby(level='stock').log_pr_3.apply(std_10)

In [31]:
def adr(log_pr, window=10, min_periods=10):
    high = log_pr.rolling(window=window, min_periods=min_periods).max()
    low = log_pr.rolling(window=window, min_periods=min_periods).min()

    # average over window
    ma_high = high.rolling(window=window).mean()
    ma_low = low.rolling(window=window).mean()

    ratio = ma_high/ma_low

    return 100 * (ratio - 1)
    

In [34]:
features['pr_adr_20'] = features.groupby(level='stock').log_pr.apply(adr)

### z score of volume

In [35]:
# roll_rank_fn = lambda x: x.rolling(240, min_period=20).apply(lambda x: pd.Series(x).rank(pct=True)[0])
zscore_fn = lambda x: (x - x.rolling(window=240, min_periods=20).mean()) / x.rolling(window=240, min_periods=20).std()
features['volu_z_score'] = volu_df.groupby(level='stock').volu.apply(zscore_fn)

In [36]:
volu_df.groupby(level='stock').volu.rolling(200, min_periods=20).max()

stock  timestamp            stock
0      2021-07-01 00:00:00  0                 NaN
       2021-07-01 00:01:00  0                 NaN
       2021-07-01 00:02:00  0                 NaN
       2021-07-01 00:03:00  0                 NaN
       2021-07-01 00:04:00  0                 NaN
                                         ...     
9      2021-12-31 23:55:00  9        6.669354e+07
       2021-12-31 23:56:00  9        6.669354e+07
       2021-12-31 23:57:00  9        6.669354e+07
       2021-12-31 23:58:00  9        6.669354e+07
       2021-12-31 23:59:00  9        6.669354e+07
Name: volu, Length: 2649600, dtype: float64

### Transform back to wide format

In [37]:
feature_df = features.reset_index(level=['stock']).sort_index()

In [41]:
temp = feature_df.pivot(columns ='stock')
temp.columns = temp.columns.get_level_values(0) + '_' +  [str(x) for x in temp.columns.get_level_values(1)]

In [43]:
temp.iloc[30:].isnull().any().all()

False

### Organize Code

In [216]:
def remove_outliers(dta):
    # Compute the mean and interquartile range
    mean = dta.mean(0)
    iqr = dta.quantile([0.25, 0.75], axis=0).diff().T.iloc[:, 1]
    
    # Replace entries that are more than 10 times the IQR
    # away from the mean with NaN (denotes a missing entry)
    mask = np.abs(dta) > mean + 10 * iqr
    treated = dta.copy()
    treated[mask] = np.nan

    return treated

In [48]:
def wide_format(df):
    df_= df.reset_index(level=['stock']).sort_index()
    df_ = df_.pivot(columns ='stock')
    df_.columns = df_.columns.get_level_values(0) + '_' +  [str(x) for x in df_.columns.get_level_values(1)]

    return df_


def get_feature_train(log_pr, volu, x_begin_idx, x_end_idx, y_begin_idx, 
                        grp_idx=None, rm_outlier=False, print_cor=True):
    """
    Input:
    log_pr (pdSeries): train set
    volu (pdSeries): train set
    x_begin_idx (pdIndex): to truncate the NaNs
    grp_idx (dict): key is group idx, value is list of stock idx

    Returns:
    feature_dict (dict): key is group idx, value is a tuple of feature matrix and response
    """

    log_pr_df = log_pr.reset_index().melt(id_vars=['timestamp'])
    log_pr_df.columns = ['timestamp', 'stock', 'log_pr']
    log_pr_df = log_pr_df.set_index(['timestamp', 'stock']).sort_index()

    volu_df = volu.reset_index().melt(id_vars=['timestamp'])
    volu_df.columns = ['timestamp', 'stock', 'volu']
    volu_df = volu_df.set_index(['timestamp', 'stock']).sort_index()

    features = pd.DataFrame(index=log_pr_df.index)

    # log_pr feature
    for i in [3, 10, 20, 30]:
        features['log_pr_{}'.format(i)] = -log_pr_df.groupby(level='stock').log_pr.diff(i)

    # # EMA
    # ema = lambda x: x.ewm(span=i).mean()
    # for i in [10, 30, 50]:
    #     features['pr_ema_{}'.format(i)] = log_pr_df.groupby(level='stock').log_pr.apply(ema)

    # MA
    for i in [10, 30, 50]:
        features['pr_ema_{}'.format(i)] = log_pr_df.groupby(level='stock').rolling(i).mean()

    std_10 = lambda x: x.rolling(10).std()
    features['log_pr_std_10'] = log_pr_df.groupby(level='stock').log_pr.apply(std_10)

    # RSI
    features['rsi_20'] = log_pr_df.groupby(level='stock').log_pr.apply(rsi)
    features['rsi_30'] = log_pr_df.groupby(level='stock').log_pr.apply(rsi, periods=30)
    # features['rsi_50'] = log_pr_df.groupby(level='stock').log_pr.apply(rsi, periods=50)
    
    # # ADR
    # features['pr_adr_20'] = log_pr_df.groupby(level='stock').log_pr.apply(adr)

    # volume feature
    log_fn = lambda x: np.log(x+1)
    features['log_volu'] = volu_df.groupby(level='stock').volu.apply(log_fn)

    # stdised volume in 2 hours backward rolling windows
    zscore_fn = lambda x: (x - x.rolling(window=240, min_periods=20).mean()) / x.rolling(window=240, min_periods=20).std()
    features['volu_z_score'] = volu_df.groupby(level='stock').volu.apply(zscore_fn)

    # feature_dropped = features.iloc[x_begin_idx:x_end_idx]
    response = log_pr.diff(30)
    # print(features.shape)
    # print(feature_dropped.shape)
    # print(response_dropped.shape)

    if grp_idx is not None:
        feature_dict = {}
        for key, idx_lis in grp_idx.items():
            feature_df_dropped = wide_format(features.loc[pd.IndexSlice[:,idx_lis],:])
            # transform back to wide format
            feature_dict[key] = (feature_df_dropped.iloc[x_begin_idx:x_end_idx], 
                                            response[idx_lis].iloc[y_begin_idx:])
        return feature_dict
    else:
        # transform back to wide format
        feature_df_dropped = wide_format(features).iloc[x_begin_idx:x_end_idx]
        # feature_df_dropped = feature_df[x_begin_idx:x_end_idx]
    
        if print_cor:
            for i in range(10):
                feature_train_0 = features.xs(i, level='stock')
                print(feature_train_0.corrwith(response[i]))

        return feature_df_dropped, response.iloc[y_begin_idx:]

In [49]:
def wide_format_test(df):
    df_= df.reset_index()
    df_ = df_.pivot(columns ='index').apply(lambda s: s.dropna().reset_index(drop=True))
    df_.columns = df_.columns.get_level_values(0) + '_' +  [str(x) for x in df_.columns.get_level_values(1)]

    return df_

def get_feature_test(log_pr, volu, grp_idx=None):
    """
    Input: 
    log_pr (pdSeries): 1 day of log pr 
    volu (pdSeries): 1 day of volume

    Output:
    test data frame
    """
    features = pd.DataFrame(index=log_pr.columns)

    # backward return
    # print(-(log_pr.iloc[-1] - log_pr.iloc[-30]).values)
    for i in [10, 20, 30]:
        features['log_pr_{}'.format(i)] = -(log_pr.iloc[-1] - log_pr.iloc[-i]).values
    # backward rolling std
    features['log_pr_std_10'] = log_pr.iloc[-10:].std(0).values
    
    # volume features
    features['log_volu'] = np.log(volu.iloc[-1].values + 1)
    features['volu_z_score'] = ((volu.iloc[-1] - volu.iloc[-240:].mean())/volu.iloc[-240:].std()).values

    if grp_idx is None:
        return wide_format_test(features)
    else:
        df_dict = {}
        for key, idx_lis in grp_idx.items():
            df_dict[key] = wide_format_test(features.loc[idx_lis])
        return df_dict


In [50]:
       # if rm_outlier:
        #     feature_df_dropped = remove_outliers(feature_df)
        #     na_idx = feature_df_dropped.isnull().any(1)
        #     response_dropped[na_idx] = np.nan

        #     return feature_df_dropped.dropna(), response_dropped.dropna()

In [51]:
t_train = log_pr.index[-87841]
log_pr_train = log_pr[:t_train]
volu_train = volu[:t_train]

x_begin_idx = 30
x_end_idx = -30
y_begin_idx = 60

# grp_idx = {0:[1,5,6,8], 1:[2,4,7], 2:[0,3,9]}
feature_dict = get_feature_train(log_pr_train, volu_train, x_begin_idx, x_end_idx, y_begin_idx, grp_idx=None)

log_pr_3        -0.309250
log_pr_10       -0.566440
log_pr_20       -0.807458
log_pr_30       -1.000000
pr_ema_10        0.002535
pr_ema_30       -0.006325
pr_ema_50       -0.011227
log_pr_std_10    0.294352
rsi_20           0.754775
rsi_30           0.772893
rsi_50           0.738094
pr_adr_20       -0.001367
log_volu         0.137962
volu_z_score     0.157233
dtype: float64
log_pr_3        -0.303509
log_pr_10       -0.564339
log_pr_20       -0.810175
log_pr_30       -1.000000
pr_ema_10        0.000539
pr_ema_30       -0.006753
pr_ema_50       -0.010747
log_pr_std_10    0.353098
rsi_20           0.657330
rsi_30           0.680200
rsi_50           0.661198
pr_adr_20       -0.002536
log_volu         0.138490
volu_z_score     0.188771
dtype: float64
log_pr_3        -0.306207
log_pr_10       -0.566170
log_pr_20       -0.811186
log_pr_30       -1.000000
pr_ema_10        0.000290
pr_ema_30       -0.004223
pr_ema_50       -0.006759
log_pr_std_10    0.308044
rsi_20           0.730604
rsi_30  